In [62]:
import requests
import osmnx as ox
import json
import geopandas as gpd
from shapely.geometry import mapping
import os
import numpy as np
from pyproj import Transformer

In [ ]:
TRANSFORMER_DEG = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
APIKEY = "my key"
ZONE = "Paris, France"
ROUNDABOUTS = None
PHOTO_MAX_RONDPOINT = 21
RAYON = 1000
RAYON_DISCRIMINANT = 50 # en metres zone autour d'une coordonnée sans autre coordonnée


ROUNDABOUTS = ox.features_from_place(ZONE, tags={'junction': 'roundabout'})

ROUNDABOUTS = ROUNDABOUTS.to_crs(epsg=3857)

ROUNDABOUTS['center'] = ROUNDABOUTS.centroid


print(ROUNDABOUTS['center'])

ROUNDABOUTS = ROUNDABOUTS.reset_index(drop=True)
ROUNDABOUTS['id'] = (ROUNDABOUTS.index).astype(int)

ROUNDABOUTS = ROUNDABOUTS[['id', 'center', 'geometry']]

element  id        
way      11996681      POINT (249558.518 6249148.401)
         24337185      POINT (258943.906 6247452.656)
         130584045     POINT (249638.024 6249099.115)
         168743946     POINT (249662.162 6249149.413)
         168743950     POINT (249615.664 6249181.604)
         879299906      POINT (249576.37 6249166.914)
         1025542374    POINT (249611.043 6249083.326)
Name: center, dtype: geometry


In [ ]:
def pointgen(centre, rayon, angle):
    x = centre[0] + (rayon * np.cos(angle))
    y = centre[1] + (rayon * np.sin(angle))
    point = (x, y)
    return point

In [ ]:
coordonnees = np.zeros((len(ROUNDABOUTS), PHOTO_MAX_RONDPOINT, 2), float)

distance_nombre = RAYON // 2*RAYON_DISCRIMINANT

angle_distance_possible = []

for i in ROUNDABOUTS['id']:
    centre = ROUNDABOUTS['center'][i]
    coordonnees[i, 0, 0], coordonnees[i, 0, 1] = TRANSFORMER_DEG.transform((ROUNDABOUTS['center'][i].x), (ROUNDABOUTS['center'][i].y))



In [73]:
features = []
for i, (_, row) in enumerate(ROUNDABOUTS.iterrows()):
    slice_coords = coordonnees[i]
    poly_list = slice_coords.tolist()
    features.append({
        "type": "Feature",
        "id": int(row['id']),
        "properties": {
            "center": [TRANSFORMER_DEG.transform(row['center'].x, row['center'].y)]
        },
        "geometry": {
        "type": "Polygon",
        "coordinates": [poly_list]
        }
    })

geojson = {"type": "FeatureCollection", "features": features}

with open("roundabouts.json", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print(f"Nombre de ronds-points trouvés : {len(ROUNDABOUTS)}")


Nombre de ronds-points trouvés : 7


In [35]:
def recup_image_google_street_view(lat, lon, heading=0, fov=90, pitch=0, size="600x400", api_key=APIKEY):
    return f"https://maps.googleapis.com/maps/api/streetview?size={size}&location={lat},{lon}&fov={fov}&heading={heading}&pitch={pitch}&key={api_key}"

In [36]:
def recup_from_coord(lat, long, folder_name, id):
    image_path = os.path.join(folder_name, f"streetview_{id}.jpg")
    url = recup_image_google_street_view(lat, long)
    response = requests.get(url)
    if response.status_code == 200:
        with open(image_path, "wb") as img_file:
            img_file.write(response.content)
    else:
        print(f"Erreur lors du téléchargement pour le rond-point {id}")

In [51]:
for elem in geojson["features"]:
    id = elem["id"]
    geometry = elem["geometry"]
    folder_name = f"roundabout_{id}"
    os.makedirs(folder_name, exist_ok=True)

    if geometry['type'] == 'Polygon':
        polygone_coords = geometry['coordinates'][0]
    else:
        polygone_coords = geometry['coordinates']
        
    for i, coord in enumerate(polygone_coords):
        lat = coord[1]
        long = coord[0]
        recup_from_coord(lat, long, folder_name, i)